# Spatial image filtering: convolutions

In [297]:
import numpy as np
import imageio 
import matplotlib.pyplot as plt

### 2D Convolution

The 2D convolution operation is a fundamental building block in image processing. It is used to apply a filter to an image, which can be used to perform tasks such as blurring, sharpening, edge detection, and noise reduction. In this notebook, we will explore the 2D convolution operation and its applications.

### 2D Cross-correlation

The 2D cross-correlation operation is similar to the 2D convolution operation, and is used to apply a filter to an image. The main difference between the two operations is that the 2D convolution operation uses a filter that is rotated by 180 degrees, while the 2D cross-correlation operation does not. In this notebook, we will explore the 2D cross-correlation operation and its applications.

In [298]:
np.random.seed(666) # defining a seed

# random image with 5x5 size
f = np.random.randint(0, 7, [5, 5])
print(f)

[[ 25  50   1   4]
 [255  52   2   5]
 [255 100   0   3]
 [255 100   3 120]]


In [299]:
# arbitrary filter with 3x3 size
w = np.matrix([[1, 2, 0], [1, 4, 0], [0, 0, 0]]) / 8.0 # soma do filtro = 8, mas queremos que ela seja igual a 1
print(w)

[[ 1  1  1]
 [ 1 -8  1]
 [ 1  1  1]]
[[0.  0.1 0. ]
 [0.1 0.6 0.1]
 [0.  0.1 0. ]]


In [327]:
wf = np.flip(np.flip(w, 0), 1) # flip the filter
print(wf)

[[ 1  1  1]
 [ 1 -8  1]
 [ 1  1  1]]


$g(1,2) = w(x,y) * f(1,2) = w_f(x,y) (*) f(1,2)$

where $w(x,y)$ is the filter, $f(1,2)$ is the input image, and $g(1,2)$ is the output image.

In [331]:
# compute output value of g(1,2)
x, y = 2, 1

# slice the matrix in the region needed for the filtering
# I know the filter has size 3x3: x-1 to x+1 and y-1 to y+1
print(f[x-1:x+2, y-1:y+2])

[[255  52   2]
 [255 100   0]
 [255 100   3]]


In [335]:
mult_1_2 = np.multiply(f[x-1:x+2, y-1:y+2], wf)
print(mult_1_2)

102.44444444444444
[[ 255   52    2]
 [ 255 -800    0]
 [ 255  100    3]]


In [333]:
# sum all multiplied values
g_1_2 = np.sum(mult_1_2)
print(g_1_2)

122


In [304]:
# a function that performs convolution at some pixel x, y coordinates
def conv_point(f, w, x, y, debug = False):
    '''
    Performs convolution at x, y coordinates
    parameters:
        f - input image
        w - filter
        x - x coordinate
        y - y coordinate
    '''
    # compute the range of indices a, b convolution
    n, m = w.shape # size of the filter
    a = int((n-1)/2.0)
    b = int((m-1)/2.0)

    # get submatrix of a pixel neighborhood
    sub_f = f[x-a:x+a+1, y-b:y+b+1] 

    # flip the original filter
    wf = np.flip(np.flip(w, 0), 1)

    if(debug == True):
        print("sub-image f:\n" + str(sub_f))
        print("flipped filter w:\n" + str(wf))

    value = np.sum(np.multiply(sub_f, wf))

    return value

In [305]:
conv_point(f, w, 1, 2, debug = True)

sub-image f:
[[ 50   1   4]
 [ 52   2   5]
 [100   0   3]]
flipped filter w:
[[0.    0.    0.   ]
 [0.    0.5   0.125]
 [0.    0.25  0.125]]


2.0

In [306]:
# function to perform convolution on the whole image
def conv_image(f, w):
    '''
    Performs convolution on the whole image
    parameters:
        f - input image
        w - filter
    '''
    n, m = w.shape # size of the filter
    a = int((n-1)/2.0)
    b = int((m-1)/2.0)

    N, M = f.shape

    # create a new empty image to store the output values
    g = np.zeros(f.shape, dtype = np.uint8)

    # flip the original filter
    wf = np.flip(np.flip(w, 0), 1)

    # loop through all pixels of the image, not considering the borders
    for x in range(a, N - a):
        for y in range(b, M - b):
            sub_f = f[x-a:x+a+1, y-b:y+b+1] 
            g[x,y] = np.sum(np.multiply(sub_f, wf))

    return g.astype(np.uint8)

In [307]:
conv_image(f, w)

array([[ 0,  0,  0,  0],
       [ 0, 51,  2,  0],
       [ 0, 75, 16,  0],
       [ 0,  0,  0,  0]], dtype=uint8)

In [308]:
# function to perform convolution on the whole image
def conv_image_copy(f, w):
    '''
    Performs convolution on the whole image
    parameters:
        f - input image
        w - filter
    '''
    n, m = w.shape # size of the filter
    a = int((n-1)/2.0)
    b = int((m-1)/2.0)

    N, M = f.shape

    # create a new empty image to store the output values
    g = np.array(f, copy = True)

    # flip the original filter
    wf = np.flip(np.flip(w, 0), 1)

    # loop through all pixels of the image, not considering the borders
    for x in range(a, N - a):
        for y in range(b, M - b):
            sub_f = f[x-a:x+a+1, y-b:y+b+1] 
            g[x,y] = np.sum(np.multiply(sub_f, wf))

    return g.astype(np.uint8)

In [309]:
conv_image_copy(f, w)

array([[ 25,  50,   1,   4],
       [255,  51,   2,   5],
       [255,  75,  16,   3],
       [255, 100,   3, 120]], dtype=uint8)

### Zero padding

Zero padding is a technique used to preserve the size of the input image when applying a filter. It involves adding zeros around the edges of the input image, which allows the filter to be applied to the entire image. 

In [310]:
# function to perform convolution on the whole image
def conv_image(f, w, zero_padding = False):
    '''
    Performs convolution on the whole image
    parameters:
        f - input image
        w - filter
    '''
    n, m = w.shape # size of the filter
    a = int((n-1)/2.0)
    b = int((m-1)/2.0)

    N, M = f.shape

    g = np.array(f, copy = True)

    if zero_padding == True:
        # padding the image with zeros
        f = np.pad(f, ((a, a), (b, b)), 'constant', constant_values = (0, 0))

    # flip the original filter
    wf = np.flip(np.flip(w, 0), 1)

    # loop through all pixels of the image, not considering the borders
    for x in range(a, N - a):
        for y in range(b, M - b):
            sub_f = f[x-a:x+a+1, y-b:y+b+1] 
            g[x,y] = np.sum(np.multiply(sub_f, wf))

    return g.astype(np.uint8)

In [311]:
conv_image(f, w, zero_padding = True)

array([[ 25,  50,   1,   4],
       [255,  89,  38,   5],
       [255, 210,  51,   3],
       [255, 100,   3, 120]], dtype=uint8)